In [36]:
import sqlite3
import os
import tkinter as tk
from tkinter import filedialog
import xml.etree.ElementTree as ET
from datetime import datetime

# --- 1) Let user select folder containing the Session XML file ---
root = tk.Tk()
root.withdraw()  # Hide the root window
selected_folder = filedialog.askdirectory(initialdir='D:/Tramp Test/Data/')
if not selected_folder:
    raise ValueError("No folder was selected.")

# --- 2) Extract the test_date from the selected folder name ---
folder_name = os.path.basename(selected_folder)
test_date = folder_name.split('_', 1)[0]  # e.g., extract '2024-08-13' from '2024-08-13_105_Growth Plate_'

# --- 3) Find the Session XML file in the selected folder ---
xml_file_path = ''
for r, dirs, files in os.walk(selected_folder):
    for file in files:
        if file.lower().startswith('session') and file.lower().endswith('.xml'):
            xml_file_path = os.path.join(r, file)
            break
    if xml_file_path:
        break

if not xml_file_path:
    raise FileNotFoundError("No 'Session' XML file found in the selected folder.")

# --- 4) Parse the XML file ---
tree = ET.parse(xml_file_path)
root_xml = tree.getroot()

# --- 5) Extract required fields from XML ---
name = root_xml.find(".//Name").text
dob = root_xml.find(".//DOB").text
height = root_xml.find(".//Height").text
weight = root_xml.find(".//Weight").text
pre_post = root_xml.find(".//Pre_Post").text.lower()   # "pre" or "post"
exp_control = root_xml.find(".//Exp_Control").text.lower()  # "exp" or "control"
creation_date = root_xml.find(".//Creation_date").text
comments = root_xml.find(".//Comments").text

print("-------")
print("Parsed name =", repr(name))
print("Parsed dob =", repr(dob))
print("Parsed pre_post =", repr(pre_post))
print("Parsed exp_control =", repr(exp_control))
print("Parsed creation_date =", repr(creation_date))

# --- 6) Calculate age from DOB (optional, only if you need it) ---
# (not strictly necessary, but left here as an example)
dob_date = datetime.strptime(dob, "%Y-%m-%d")
today = datetime.today()
age = today.year - dob_date.year - ((today.month, today.day) < (dob_date.month, dob_date.day))

# --- 7) Connect to (and/or create) the new SQLite database ---
db_path = 'D:/Tramp Test/Tramp_Test2.sqlite'
conn = sqlite3.connect(db_path)

# --- 8) Create a single table to store all data ---
# We will store everything as requested in one table.
conn.execute('''
    CREATE TABLE IF NOT EXISTS TrampolineData (
        name TEXT,        -- The participant's name (unique enough per your request)
        dob DATE,
        height REAL,
        weight REAL,
        test_date DATE,
        pre_post TEXT,    -- "pre" or "post"
        exp_control TEXT, -- "exp" or "control"
        comments TEXT,
        trial_num INTEGER,
        movement TEXT
    )
''')

cursor = conn.cursor()

# --- 9) Insert the participant (and “session”-level data) ---
# We'll do an "upsert"-like pattern, but since you want to treat 'name' as the unique participant ID,
# we will not block insertion if the participant already exists. Instead, each day’s testing is
# appended as new rows with the given name and test_date.

# For each "movement" file found, we’ll read the lines and create multiple rows in TrampolineData.
movements = ['cmj', 'dj', 'ppu']
for movement in movements:
    file_path = os.path.join(selected_folder, f"{movement}.txt")
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            lines = file.readlines()
            # Start reading from line 6 (index 5); the first 5 lines are presumably headers
            data_lines = lines[5:]
            # For each trial, insert one row into our single table
            for trial_idx, _ in enumerate(data_lines, start=1):
                cursor.execute('''
                    INSERT INTO TrampolineData (
                        name, dob, height, weight,
                        test_date, pre_post, exp_control,
                        comments, trial_num, movement
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    name,
                    dob,
                    height,
                    weight,
                    test_date,
                    pre_post,
                    exp_control,
                    comments,
                    trial_idx,
                    movement
                ))
    else:
        # If the file doesn't exist, you can choose to ignore or print a warning
        print(f"Warning: {movement}.txt not found in the folder. No data inserted for {movement}.")

# If you want to insert at least one row even if no movement file is found,
# you could add that logic here. Currently, we only insert if the text file is found.

# --- 10) Commit and close the DB connection ---
conn.commit()
conn.close()

# --- 11) Print a completion message ---
print(f"Data for participant '{name}' on test date '{test_date}' has been inserted into 'Tramp_Test2.sqlite'.")


-------
Parsed name = 'Cranz Smelcer'
Parsed dob = '2006-03-07'
Parsed pre_post = 'post'
Parsed exp_control = 'control'
Parsed creation_date = '2025-06-11'
Data for participant 'Cranz Smelcer' on test date '2025-06-11' has been inserted into 'Tramp_Test2.sqlite'.


In [37]:
# New code block of the second cell to try and remedy the  issues of the previous cell 2 versions

import re
import pandas as pd
import sqlite3
import os

def extract_test_date_from_ascii(ascii_file_path: str) -> str:
    """
    Extracts the test date in 'YYYY-MM-DD' format from the first file path in the ASCII file.
    """
    with open(ascii_file_path, 'r') as file:
        lines = file.readlines()
        # Extract the first file path from the first line
        first_file_path = lines[0].strip().split('\t')[0]
        parts = first_file_path.split('\\')
        if len(parts) > 4:
            date_folder = parts[4]  # e.g. "2025-01-02__2"
            match = re.match(r'^\d{4}-\d{2}-\d{2}', date_folder)
            if match:
                return match.group(0)
            else:
                raise ValueError(f"Unable to extract test date from folder: {date_folder}")
        else:
            raise ValueError("Unexpected file path structure: Unable to extract test date.")

# 1) Verify that global variables from the first code cell (name, dob, etc.) are available
required_globals = ['name', 'dob', 'height', 'weight', 'exp_control', 'comments', 'pre_post']
missing_globals = [g for g in required_globals if g not in globals()]
if missing_globals:
    raise ValueError(
        f"The following variables are missing from the global scope: {missing_globals}. "
        "Please ensure the first code block was executed (and completed successfully) first."
    )

# 2) Connect to the single-table database (Tramp_Test2.sqlite)
db_path = 'D:/Tramp Test/Tramp_Test2.sqlite'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 3) Ensure our single table has the extra metric columns we need
#    If they don't exist, we try to add them. (No harm if columns already exist.)
additional_columns = [
    ('JH_IN', 'REAL'),
    ('LEWIS_PEAK_POWER', 'REAL'),
    ('NORM_LEWIS_PEAK_POWER_KG', 'REAL'),
    ('Max_Force', 'REAL'),
]
for col_name, col_type in additional_columns:
    try:
        cursor.execute(f"ALTER TABLE TrampolineData ADD COLUMN {col_name} {col_type};")
    except sqlite3.OperationalError:
        pass  # This means the column already exists

# 4) Directory where the ASCII files are located
ascii_dir = 'D:/Tramp Test/Output Files/'

# 5) Process each movement file
movements = ['cmj', 'dj', 'ppu']
for movement in movements:
    file_path = os.path.join(ascii_dir, f"{movement}.txt")
    if not os.path.exists(file_path):
        print(f"File not found for {movement} at: {file_path}")
        continue

    # Extract the test date from the ASCII file
    test_date = extract_test_date_from_ascii(file_path)
    print(f"\n>>> Processing {movement.upper()} | Test date: {test_date}")

    with open(file_path, 'r') as file:
        lines = file.readlines()

    print(f"Total lines read for {movement}: {len(lines)}")

    # According to your example, line[1] (index 1) might have metric headers,
    # and real data often starts around line 6 (index=5).
    data_start_index = 5

    # If your second line (index=1) has the metric header, let's just read it quickly:
    raw_header = lines[1].strip().split('\t')
    if len(raw_header) > 0 and raw_header[0].strip() == '':
        raw_header = raw_header[1:]  # remove first empty column if present
    print(f"Header (raw): {raw_header}")

    # Slice out the data lines
    all_data_rows = [line.strip().split('\t') for line in lines[data_start_index:]]

    # Each row might contain multiple trials of 4 metrics each
    for row_idx, row_data in enumerate(all_data_rows, start=data_start_index):
        if not row_data or len(row_data) < 5:
            continue  # skip empty / short lines

        # First column might be an "Item" label; the rest are the metric groups
        item_number = row_data[0]  
        metric_cols = row_data[1:]
        num_metrics = 4  # JH_IN, LEWIS_PEAK_POWER, NORM_LEWIS_PEAK_POWER_KG, Max_Force

        if len(metric_cols) % num_metrics != 0:
            print(f"Warning: row {row_idx} has {len(metric_cols)} cols (not multiple of {num_metrics}).")

        num_trials = len(metric_cols) // num_metrics

        # Loop over each trial chunk
        for trial_i in range(num_trials):
            start_index = trial_i * num_metrics
            end_index   = start_index + num_metrics

            jh_in                 = metric_cols[start_index]   if start_index+0 < len(metric_cols) else None
            lewis_peak_power      = metric_cols[start_index+1] if start_index+1 < len(metric_cols) else None
            norm_lewis_peak_power = metric_cols[start_index+2] if start_index+2 < len(metric_cols) else None
            max_force             = metric_cols[start_index+3] if start_index+3 < len(metric_cols) else None

            trial_num = trial_i + 1

            # Debug
            print(f"  Trial #{trial_num} => JH_IN={jh_in}, LEWIS={lewis_peak_power}, "
                  f"NORM={norm_lewis_peak_power}, FORCE={max_force}")

            # Check if we already have a placeholder (or existing entry) in the single table
            # for this participant/test_date/movement/trial_num. If so, UPDATE. Otherwise, INSERT.
            cursor.execute("""
                SELECT rowid
                FROM TrampolineData
                WHERE name = ?
                  AND test_date = ?
                  AND movement = ?
                  AND pre_post = ?
                  AND trial_num = ?
            """, (name, test_date, movement, pre_post, trial_num))
            existing_row = cursor.fetchone()

            if existing_row:
                # Update existing
                cursor.execute("""
                    UPDATE TrampolineData
                    SET JH_IN = ?,
                        LEWIS_PEAK_POWER = ?,
                        NORM_LEWIS_PEAK_POWER_KG = ?,
                        Max_Force = ?
                    WHERE rowid = ?;
                """, (
                    jh_in,
                    lewis_peak_power,
                    norm_lewis_peak_power,
                    max_force,
                    existing_row[0]
                ))
            else:
                # Insert new row
                cursor.execute("""
                    INSERT INTO TrampolineData (
                        name, dob, height, weight,
                        test_date, pre_post, exp_control,
                        comments, trial_num, movement,
                        JH_IN, LEWIS_PEAK_POWER,
                        NORM_LEWIS_PEAK_POWER_KG, Max_Force
                    )
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
                """, (
                    name,
                    dob,
                    height,
                    weight,
                    test_date,
                    pre_post,
                    exp_control,
                    comments,
                    trial_num,
                    movement,
                    jh_in,
                    lewis_peak_power,
                    norm_lewis_peak_power,
                    max_force
                ))

# 6) Commit changes
conn.commit()
conn.close()

print("\nAll trials processed and inserted/updated in 'TrampolineData' (single-table schema).")



>>> Processing CMJ | Test date: 2025-06-11
Total lines read for cmj: 6
Header (raw): ['JH_IN', 'LEWIS_PEAK_POWER', 'NORM_LEWIS_PEAK_POWER_KG', 'Max_Force', 'JH_IN', 'LEWIS_PEAK_POWER', 'NORM_LEWIS_PEAK_POWER_KG', 'Max_Force']
  Trial #1 => JH_IN=17.19, LEWIS=4560.30, NORM=4560.30, FORCE=5416.68
  Trial #2 => JH_IN=16.43, LEWIS=4440.87, NORM=4440.87, FORCE=5854.35

>>> Processing DJ | Test date: 2025-06-11
Total lines read for dj: 6
Header (raw): ['JH_IN', 'LEWIS_PEAK_POWER', 'NORM_LEWIS_PEAK_POWER_KG', 'Max_Force', 'JH_IN', 'LEWIS_PEAK_POWER', 'NORM_LEWIS_PEAK_POWER_KG', 'Max_Force']
  Trial #1 => JH_IN=18.96, LEWIS=4838.87, NORM=4838.87, FORCE=5855.80
  Trial #2 => JH_IN=17.77, LEWIS=4651.64, NORM=4651.64, FORCE=5853.13

>>> Processing PPU | Test date: 2025-06-11
Total lines read for ppu: 6
Header (raw): ['JH_IN', 'LEWIS_PEAK_POWER', 'NORM_LEWIS_PEAK_POWER_KG', 'Max_Force', 'JH_IN', 'LEWIS_PEAK_POWER', 'NORM_LEWIS_PEAK_POWER_KG', 'Max_Force']
  Trial #1 => JH_IN=2.41, LEWIS=2236.60, 

In [38]:
# Reorders the database to be in alphabetical order

import sqlite3


db_path = "D:/Tramp Test/Tramp_Test.sqlite" 
sort_column = "name"     

def reorder_all_tables(db_path, sort_column):
    try:
        # Connect to the database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Fetch all table names in the database
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()

        for table in tables:
            table_name = table[0]

            # Skip system tables like sqlite_sequence
            if table_name.startswith("sqlite_"):
                continue

            print(f"Processing table: {table_name}")

            # Check if the column exists in the current table
            cursor.execute(f"PRAGMA table_info({table_name});")
            columns = [info[1] for info in cursor.fetchall()]
            if sort_column not in columns:
                print(f"Skipping table '{table_name}' - Column '{sort_column}' not found.")
                continue

            # Create a new sorted table
            temp_table = f"{table_name}_sorted"
            cursor.execute(f"CREATE TABLE {temp_table} AS SELECT * FROM {table_name} ORDER BY {sort_column} ASC;")
            
            # Drop the old table
            cursor.execute(f"DROP TABLE {table_name};")
            
            # Rename the new table to the original name
            cursor.execute(f"ALTER TABLE {temp_table} RENAME TO {table_name};")
            print(f"Table '{table_name}' reordered successfully.")

        # Commit changes
        conn.commit()
        print("All tables processed.")
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()

reorder_all_tables(db_path, sort_column)


Processing table: Sessions
Skipping table 'Sessions' - Column 'name' not found.
Processing table: Movements
Skipping table 'Movements' - Column 'name' not found.
Processing table: Participants
Table 'Participants' reordered successfully.
Processing table: Results
Table 'Results' reordered successfully.
All tables processed.


In [39]:
import sqlite3
import pandas as pd
import numpy as np
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html, Input, Output
import plotly.graph_objects as go

# ------------------------------------------------
# 1) Data Loading
# ------------------------------------------------
db_path = r'D:/Tramp Test/Tramp_Test2.sqlite'
conn = sqlite3.connect(db_path)

# Single table schema:
query = """
SELECT
    name,
    movement,
    pre_post,
    test_date,
    JH_IN AS jump_height,
    LEWIS_PEAK_POWER AS peak_power
FROM TrampolineData
WHERE movement IN ('cmj', 'dj', 'ppu')
"""
df = pd.read_sql_query(query, conn)
conn.close()

# Convert test_date to datetime
df['test_date'] = pd.to_datetime(df['test_date'], errors='coerce')

# Drop rows without participant name or pre_post
df.dropna(subset=['name', 'pre_post'], inplace=True)

# Gather participant names
participants = sorted(df['name'].dropna().unique())


# ------------------------------------------------
# 2) Dash App Initialization w/ Cyborg (dark theme)
# ------------------------------------------------
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])
app.title = "Jump Data Dashboard"


# ------------------------------------------------
# 3) Reusable Components / Helper Functions
# ------------------------------------------------
def build_pre_post_figure(df_filtered, movement_label, y_col):
    """
    movement_label: 'cmj', 'dj', 'ppu'
    y_col: 'jump_height' or 'peak_power'

    Returns a Figure with:
      - All raw points for pre/post, displayed as hollow/bold circles
      - A dashed line connecting average(pre) -> average(post)
      - One color per test_date
    """
    sub_df = df_filtered[df_filtered['movement'] == movement_label].copy()
    sub_df.sort_values('test_date', inplace=True)  # consistent color ordering

    unique_dates = sub_df['test_date'].dropna().unique()
    fig = go.Figure()

    for session_date in unique_dates:
        sess_data = sub_df[sub_df['test_date'] == session_date]
        if sess_data.empty:
            continue

        # Split out pre/post
        pre_data = sess_data[sess_data['pre_post'].str.lower() == 'pre']
        post_data = sess_data[sess_data['pre_post'].str.lower() == 'post']

        # Plot raw points for PRE (hollow circle markers)
        if not pre_data.empty:
            fig.add_trace(go.Scatter(
                x=['pre']*len(pre_data),
                y=pre_data[y_col],
                mode='markers',
                marker=dict(
                    symbol='circle-open',
                    line=dict(width=2),
                    size=8
                ),
                name=f"{session_date.date()} (pre)",
                opacity=0.9,
            ))

        # Plot raw points for POST (hollow circle markers)
        if not post_data.empty:
            fig.add_trace(go.Scatter(
                x=['post']*len(post_data),
                y=post_data[y_col],
                mode='markers',
                marker=dict(
                    symbol='circle-open',
                    line=dict(width=2),
                    size=8
                ),
                name=f"{session_date.date()} (post)",
                opacity=0.9,
            ))

        # Dashed line from mean(pre) to mean(post)
        if not pre_data.empty and not post_data.empty:
            avg_pre = pre_data[y_col].mean()
            avg_post = post_data[y_col].mean()
            if not np.isnan(avg_pre) and not np.isnan(avg_post):
                fig.add_trace(go.Scatter(
                    x=['pre', 'post'],
                    y=[avg_pre, avg_post],
                    mode='lines',
                    line=dict(dash='dash', width=2),
                    opacity=0.9,
                    showlegend=False
                ))

    fig.update_layout(
        paper_bgcolor='#212529',   # outside the plot (dark greyish)
        plot_bgcolor='#343a40',    # behind the data
        font=dict(color='white'),
        xaxis=dict(title="Pre vs Post", type='category'),
        yaxis=dict(title=y_col.replace('_',' ').title()),
        hovermode='closest',
        margin=dict(l=40, r=40, t=60, b=40),
    )
    return fig


def build_summary_cards(dff):
    """
    Create cards showing both average and max jump height/power,
    comparing pre vs. post.
    """
    if dff.empty:
        return dbc.Row([
            dbc.Col(
                dbc.Card(
                    dbc.CardBody([
                        html.H5("No Data Found", className="card-title text-white"),
                        html.P("This participant has no records.", className="text-white")
                    ]),
                    color="dark",
                    inverse=True
                ),
                width=12
            )
        ])

    # Separate pre vs. post
    pre_df = dff[dff['pre_post'].str.lower() == 'pre']
    post_df = dff[dff['pre_post'].str.lower() == 'post']

    # Compute means and max
    avg_jh_pre = pre_df['jump_height'].mean() if not pre_df.empty else None
    avg_jh_post = post_df['jump_height'].mean() if not post_df.empty else None
    max_jh_pre = pre_df['jump_height'].max() if not pre_df.empty else None
    max_jh_post = post_df['jump_height'].max() if not post_df.empty else None

    avg_pw_pre = pre_df['peak_power'].mean() if not pre_df.empty else None
    avg_pw_post = post_df['peak_power'].mean() if not post_df.empty else None
    max_pw_pre = pre_df['peak_power'].max() if not pre_df.empty else None
    max_pw_post = post_df['peak_power'].max() if not post_df.empty else None

    # Helper: format float or show "N/A"
    def fmt(x):
        return f"{x:.2f}" if (x is not None and not np.isnan(x)) else "N/A"

    # Two cards side by side
    return dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Jump Height (in)", className="text-white bg-secondary"),
                dbc.CardBody([
                    html.Div([
                        html.Span("Avg Pre: ", style={"fontWeight": "bold"}),
                        html.Span(fmt(avg_jh_pre))
                    ], className="text-white"),
                    html.Div([
                        html.Span("Avg Post: ", style={"fontWeight": "bold"}),
                        html.Span(fmt(avg_jh_post))
                    ], className="text-white mt-1"),
                    html.Hr(style={"backgroundColor": "white"}),
                    html.Div([
                        html.Span("Max Pre: ", style={"fontWeight": "bold"}),
                        html.Span(fmt(max_jh_pre))
                    ], className="text-white mt-1"),
                    html.Div([
                        html.Span("Max Post: ", style={"fontWeight": "bold"}),
                        html.Span(fmt(max_jh_post))
                    ], className="text-white mt-1"),
                ]),
            ], color="dark", inverse=True, className="mb-3"),
            width=6
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Peak Power", className="text-white bg-secondary"),
                dbc.CardBody([
                    html.Div([
                        html.Span("Avg Pre: ", style={"fontWeight": "bold"}),
                        html.Span(fmt(avg_pw_pre))
                    ], className="text-white"),
                    html.Div([
                        html.Span("Avg Post: ", style={"fontWeight": "bold"}),
                        html.Span(fmt(avg_pw_post))
                    ], className="text-white mt-1"),
                    html.Hr(style={"backgroundColor": "white"}),
                    html.Div([
                        html.Span("Max Pre: ", style={"fontWeight": "bold"}),
                        html.Span(fmt(max_pw_pre))
                    ], className="text-white mt-1"),
                    html.Div([
                        html.Span("Max Post: ", style={"fontWeight": "bold"}),
                        html.Span(fmt(max_pw_post))
                    ], className="text-white mt-1"),
                ]),
            ], color="dark", inverse=True, className="mb-3"),
            width=6
        ),
    ])


# ------------------------------------------------
# 4) Layout
# ------------------------------------------------

# 4a) Navbar
navbar = dbc.NavbarSimple(
    brand="Jump Data Dashboard",
    color="dark",
    dark=True,
    sticky="top",
    className="mb-2",
)

# 4b) Sidebar (dark)
sidebar = dbc.Col(
    [
        html.H5("Select Participant:", className="mt-3 text-white"),
        dcc.Dropdown(
            id='participant-dropdown',
            options=[{'label': p, 'value': p} for p in participants],
            value=participants[0] if participants else None,
            clearable=False,
            style={"color": "black"}
        ),
        html.Hr(style={"backgroundColor": "white"}),

        html.H5("Select Test Dates:", className="mt-3 text-white"),
        dcc.Checklist(
            id='date-checklist',
            options=[],  # We'll update dynamically
            value=[],    # We'll update dynamically
            labelStyle={'display': 'block'},
            inputStyle={"margin-right": "5px"}
        ),
        html.Hr(style={"backgroundColor": "white"}),

        html.Div(id="summary-cards"),
    ],
    width=3,
    style={
        "backgroundColor": "#212529",
        "minHeight": "100vh",
        "borderRight": "1px solid #343a40",
        "padding": "20px"
    },
)


# 4c) Tabs for CMJ, DJ, PPU each with two subplots
content = dbc.Col(
    [
        dbc.Tabs(
            [
                dbc.Tab(
                    label="CMJ",
                    tab_id="tab-cmj",
                    children=[
                        html.Br(),
                        dbc.Row([
                            dbc.Col(dcc.Graph(id='cmj-jh-graph'), width=6),
                            dbc.Col(dcc.Graph(id='cmj-power-graph'), width=6),
                        ]),
                    ],
                ),
                dbc.Tab(
                    label="DJ",
                    tab_id="tab-dj",
                    children=[
                        html.Br(),
                        dbc.Row([
                            dbc.Col(dcc.Graph(id='dj-jh-graph'), width=6),
                            dbc.Col(dcc.Graph(id='dj-power-graph'), width=6),
                        ]),
                    ],
                ),
                dbc.Tab(
                    label="PPU",
                    tab_id="tab-ppu",
                    children=[
                        html.Br(),
                        dbc.Row([
                            dbc.Col(dcc.Graph(id='ppu-jh-graph'), width=6),
                            dbc.Col(dcc.Graph(id='ppu-power-graph'), width=6),
                        ]),
                    ],
                ),
            ],
            id="movement-tabs",
            active_tab="tab-cmj",
            style={"backgroundColor": "#212529"},  # dark tab background
        )
    ],
    width=9,
    style={"backgroundColor": "#212529", "padding": "20px"}
)

# Main layout
app.layout = html.Div([
    navbar,
    dbc.Container(
        dbc.Row([
            sidebar,
            content
        ]),
        fluid=True,
        style={"maxWidth": "100%", "padding": "0px", "backgroundColor": "#212529"}
    )
])


# ------------------------------------------------
# 5) Callbacks
# ------------------------------------------------
@app.callback(
    [
        Output('cmj-jh-graph','figure'),
        Output('cmj-power-graph','figure'),
        Output('dj-jh-graph','figure'),
        Output('dj-power-graph','figure'),
        Output('ppu-jh-graph','figure'),
        Output('ppu-power-graph','figure')
    ],
    [
        Input('participant-dropdown','value'),
        Input('date-checklist','value')  # new input
    ]
)
def update_plots(selected_participant, selected_dates):
    """
    For each movement (CMJ, DJ, PPU), create two figures:
      - jump_height
      - peak_power
    Return them in the same order as the outputs.
    """
    # Filter data for participant
    dff = df[df['name'] == selected_participant].copy()

    # 'selected_dates' is a list of date strings (e.g. ["2023-01-05", "2023-01-10"])
    # Convert them back to Timestamps (or directly compare date strings if you prefer)
    # The below is a simple approach if your test_date is stored as datetime in df:
    if selected_dates:
        # Make a new column with .date() as string for easy comparison
        dff['date_str'] = dff['test_date'].dt.date.astype(str)
        dff = dff[dff['date_str'].isin(selected_dates)]

    cmj_jh_fig    = build_pre_post_figure(dff, 'cmj', 'jump_height')
    cmj_power_fig = build_pre_post_figure(dff, 'cmj', 'peak_power')
    dj_jh_fig     = build_pre_post_figure(dff, 'dj',  'jump_height')
    dj_power_fig  = build_pre_post_figure(dff, 'dj',  'peak_power')
    ppu_jh_fig    = build_pre_post_figure(dff, 'ppu', 'jump_height')
    ppu_power_fig = build_pre_post_figure(dff, 'ppu', 'peak_power')

    return (
        cmj_jh_fig,
        cmj_power_fig,
        dj_jh_fig,
        dj_power_fig,
        ppu_jh_fig,
        ppu_power_fig
    )


@app.callback(
    Output("summary-cards", "children"),
    [
        Input('participant-dropdown','value'),
        Input('date-checklist','value')
    ]
)
def update_summary_cards(selected_participant, selected_dates):
    dff = df[df['name'] == selected_participant].copy()

    # Filter by selected dates
    if selected_dates:
        dff['date_str'] = dff['test_date'].dt.date.astype(str)
        dff = dff[dff['date_str'].isin(selected_dates)]

    return build_summary_cards(dff)


@app.callback(
    Output('date-checklist', 'options'),
    Output('date-checklist', 'value'),
    Input('participant-dropdown', 'value')
)
def update_date_checklist(selected_participant):
    # Filter df for the participant
    dff = df[df['name'] == selected_participant]
    # Get unique sorted dates (dropping NaN if any)
    unique_dates = sorted(dff['test_date'].dropna().unique())

    # Convert to strings or perhaps keep them as date objects
    # but typically, checklists prefer string values 
    date_options = [
        {'label': str(d.date()), 'value': str(d.date())} 
        for d in unique_dates
    ]
    # By default, select ALL 
    selected_values = [str(d.date()) for d in unique_dates]

    return date_options, selected_values


# ------------------------------------------------
# 6) Run
# ------------------------------------------------
if __name__ == "__main__":
    app.run_server(debug=True)


In [40]:
# # Process pre/post data and visualize data
# # This code cell pulls participant demographics and creates profile in database
# 
# import sqlite3
# import os
# import tkinter as tk
# from tkinter import filedialog
# import xml.etree.ElementTree as ET
# from datetime import datetime
# 
# # Ask user to select folder containing the Session XML file
# root = tk.Tk()
# root.withdraw()  # Hide the root window
# selected_folder = filedialog.askdirectory(initialdir='D:/Tramp Test/Data/')
# if not selected_folder:
#     raise ValueError("No folder was selected.")
# 
# # Extract the test_date from the selected folder name
# folder_name = os.path.basename(selected_folder)
# test_date = folder_name.split('_', 1)[0]  # Extract '2024-08-13' from '2024-08-13_105_Growth Plate_'
# 
# # Find the XML file titled "Session" in the selected folder
# xml_file_path = ''
# for r, dirs, files in os.walk(selected_folder):
#     for file in files:
#         if file.lower().startswith('session') and file.lower().endswith('.xml'):
#             xml_file_path = os.path.join(r, file)
#             break
#     if xml_file_path:
#         break
# 
# if not xml_file_path:
#     raise FileNotFoundError("No 'Session' XML file found in the selected folder.")
# 
# # Parse the XML file
# tree = ET.parse(xml_file_path)
# root_xml = tree.getroot()
# 
# # Extract required fields from XML
# name = root_xml.find(".//Name").text
# dob = root_xml.find(".//DOB").text
# height = root_xml.find(".//Height").text
# weight = root_xml.find(".//Weight").text
# pre_post = root_xml.find(".//Pre_Post").text.lower()
# exp_control = root_xml.find(".//Exp_Control").text.lower()
# creation_date = root_xml.find(".//Creation_date").text
# comments = root_xml.find(".//Comments").text
# 
# print("-------")
# print("Parsed name =", repr(name))
# print("Parsed dob =", repr(dob))
# print("Parsed pre_post =", repr(pre_post))
# print("Parsed exp_control =", repr(exp_control))
# print("Parsed creation_date =", repr(creation_date))
# 
# # Calculate age from DOB
# dob_date = datetime.strptime(dob, "%Y-%m-%d")
# today = datetime.today()
# age = today.year - dob_date.year - ((today.month, today.day) < (dob_date.month, dob_date.day))
# 
# # Connect to the SQLite database
# db_path = 'D:/Tramp Test/Tramp_Test.sqlite'
# conn = sqlite3.connect(db_path)
# 
# # Create necessary tables
# conn.execute('''CREATE TABLE IF NOT EXISTS Participants (
#     participant_id INTEGER PRIMARY KEY AUTOINCREMENT,
#     name TEXT,
#     dob DATE,
#     height REAL,
#     weight REAL,
#     exp_control TEXT -- Simplified to exp or control
# )''')
# 
# conn.execute('''CREATE TABLE IF NOT EXISTS Sessions (
#     session_id INTEGER PRIMARY KEY AUTOINCREMENT,
#     participant_id INTEGER,
#     test_date DATE,
#     pre_post TEXT,
#     exp_control TEXT,
#     comments TEXT,
#     FOREIGN KEY (participant_id) REFERENCES Participants(participant_id)
# )''')
# 
# conn.execute('''CREATE TABLE IF NOT EXISTS Movements (
#     movement_id INTEGER PRIMARY KEY AUTOINCREMENT,
#     movement_name TEXT
# )''')
# 
# # Update Results table structure to include pre_post
# conn.execute("""
#     CREATE TABLE IF NOT EXISTS Results (
#         result_id INTEGER PRIMARY KEY AUTOINCREMENT,
#         Trial_Num INTEGER,
#         pre_post TEXT, -- pre or post
#         name TEXT,
#         movement TEXT,
#         JH_IN REAL,
#         LEWIS_PEAK_POWER REAL,
#         NORM_LEWIS_PEAK_POWER_KG REAL,
#         Max_Force REAL
#     );
# """)
# 
# # Insert participant if not already in the database
# cursor = conn.cursor()
# cursor.execute("SELECT participant_id FROM Participants WHERE name = ?", (name,))
# participant = cursor.fetchone()
# if participant is None:
#     cursor.execute("""
#         INSERT INTO Participants (name, dob, height, weight, exp_control) 
#         VALUES (?, ?, ?, ?, ?)
#     """, (name, dob, height, weight, exp_control))
#     participant_id = cursor.lastrowid
# else:
#     participant_id = participant[0]
# 
# print(f"Participant '{name}' assigned to group '{exp_control}' with participant_id: {participant_id}")
# 
# # Insert session
# cursor.execute("""
#     INSERT INTO Sessions (participant_id, test_date, pre_post, exp_control, comments) 
#     VALUES (?, ?, ?, ?, ?)
# """, (participant_id, test_date, pre_post, exp_control, comments))
# session_id = cursor.lastrowid
# 
# # Define movements and ensure they are in the Movements table
# movements = ['cmj', 'dj', 'ppu']
# movement_ids = {}
# for movement in movements:
#     cursor.execute("SELECT movement_id FROM Movements WHERE movement_name = ?", (movement,))
#     result = cursor.fetchone()
#     if result is None:
#         cursor.execute("INSERT INTO Movements (movement_name) VALUES (?)", (movement,))
#         movement_ids[movement] = cursor.lastrowid
#     else:
#         movement_ids[movement] = result[0]
# 
# # Process placeholder rows for Results table
# for movement in ['cmj', 'dj', 'ppu']:
#     file_path = os.path.join(selected_folder, f"{movement}.txt")
#     if os.path.exists(file_path):
#         # Read the file and calculate the number of trials dynamically
#         with open(file_path, 'r') as file:
#             lines = file.readlines()
#             num_trials = len(lines[5:])  # Count rows starting from line 6
#             for trial_num in range(1, num_trials + 1):
#                 cursor.execute("""
#                     INSERT INTO Results (Trial_Num, pre_post, name, movement) 
#                     VALUES (?, ?, ?, ?);
#                 """, (trial_num, pre_post, name, movement))
# 
# conn.commit()
# conn.close()
# 
# # At the end of the first code block
# global_pre_post = pre_post  # Set this as a global variable
# print(f"Global pre_post set to: {global_pre_post}")
# 
# print(f"Data for participant '{name}' with test date '{test_date}' has been inserted.")


In [41]:
# import re
# import pandas as pd
# import sqlite3
# import os
# 
# def extract_test_date_from_ascii(ascii_file_path: str) -> str:
#     """
#     Extracts the test date in 'YYYY-MM-DD' format from the first file path in the ASCII file.
#     """
#     with open(ascii_file_path, 'r') as file:
#         lines = file.readlines()
#         # Extract the first file path from the first line
#         first_file_path = lines[0].strip().split('\t')[0]
#         parts = first_file_path.split('\\')
#         if len(parts) > 4:
#             date_folder = parts[4]  # e.g. "2025-01-02__2"
#             match = re.match(r'^\d{4}-\d{2}-\d{2}', date_folder)
#             if match:
#                 return match.group(0)
#             else:
#                 raise ValueError(f"Unable to extract test date from folder: {date_folder}")
#         else:
#             raise ValueError("Unexpected file path structure: Unable to extract test date.")
# 
# 
# # Make sure the global_pre_post is available (from code cell 1)
# if 'global_pre_post' in globals():
#     pre_post = global_pre_post
#     print(f"Using global_pre_post: {pre_post}")
# else:
#     raise ValueError("global_pre_post not found. Ensure the first code block was executed.")
# 
# 
# db_path = 'D:/Tramp Test/Tramp_Test.sqlite'
# conn = sqlite3.connect(db_path)
# cursor = conn.cursor()
# 
# # Just in case the column doesn't exist in Results
# try:
#     cursor.execute("""ALTER TABLE Results ADD COLUMN Trial_Num INTEGER;""")
# except sqlite3.OperationalError:
#     pass
# 
# movements = ['cmj', 'dj', 'ppu']
# ascii_dir = 'D:/Tramp Test/Output Files/'
# 
# for movement in movements:
#     file_path = os.path.join(ascii_dir, f"{movement}.txt")
#     if not os.path.exists(file_path):
#         print(f"File not found for {movement} at: {file_path}")
#         continue
# 
#     test_date = extract_test_date_from_ascii(file_path)
#     print(f"\n>>> Processing {movement.upper()} | Test date: {test_date}")
# 
#     with open(file_path, 'r') as file:
#         lines = file.readlines()
# 
#     # Debug: how many lines
#     print(f"Total lines read for {movement}: {len(lines)}")
# 
#     # Typically, line[1] has the column names (minus an optional first col).
#     # But with your example, lines[1] has the 12 metric headers:
#     #    JH_IN, LEWIS_PEAK_POWER, NORM..., Max_Force, etc... repeated for each trial.
#     # lines[5] or lines[6] might contain the actual data row(s).
# 
#     # Let's define the start line for data
#     data_start_index = 5  # According to your example, real data is at line 6 (index=5).
# 
#     # We'll read everything from line[1] as the "header line"
#     raw_header = lines[1].strip().split('\t')
#     # If there's an initial blank or "Trial_ID" column, remove it
#     # But in your example, it might not exist. Let's check length:
#     if len(raw_header) > 0 and raw_header[0].strip() == '':
#         raw_header = raw_header[1:]  # remove the first empty column
# 
#     print(f"Header (raw): {raw_header}")
# 
#     # We'll skip lines[2], lines[3], lines[4] (METRIC, PROCESSED, ITEM, etc.).
#     # Then line[5] (index=4) might be "ITEM X X X ...", so let's jump to data_start_index=5.
# 
#     all_data_rows = [line.strip().split('\t') for line in lines[data_start_index:]]
# 
#     # Now we expect each row to have 1 + 4*N columns (where N is the number of trials in that row).
#     # In your example, we have:
#     #   13 columns total => 1 is the "Item" ID + 12 columns for 3 trials of 4 metrics each.
# 
#     for row_idx, row_data in enumerate(all_data_rows, start=data_start_index):
#         if not row_data or len(row_data) < 5:
#             # Possibly an empty line
#             continue
# 
#         # For debugging:
#         print(f"Row idx={row_idx} => {row_data}")
# 
#         # The first column in the row might be the "Item" number, e.g. "1"
#         # The rest are sets of 4 columns per trial: [JH_IN, LEWIS_PEAK, NORM_LEWIS, Max_Force]
#         item_number = row_data[0]  # Often "1"
# 
#         # Let's define the total columns for metric data
#         metric_cols = row_data[1:]  # skip the item_number
#         num_metrics = 4  # JH_IN, LEWIS, NORM_LEWIS, Max_Force
# 
#         # Figure out how many trials are in this row
#         if len(metric_cols) % num_metrics != 0:
#             print(f"Warning: row has {len(metric_cols)} metric cols which is not a multiple of 4.")
#         num_trials = len(metric_cols) // num_metrics
# 
#         # We loop over each chunk of 4 columns as a separate trial
#         for trial_i in range(num_trials):
#             start_index = trial_i * num_metrics
#             end_index   = start_index + num_metrics
# 
#             # Extract the 4 metrics
#             jh_in                 = metric_cols[start_index]   if start_index+0 < len(metric_cols) else None
#             lewis_peak_power      = metric_cols[start_index+1] if start_index+1 < len(metric_cols) else None
#             norm_lewis_peak_power = metric_cols[start_index+2] if start_index+2 < len(metric_cols) else None
#             max_force             = metric_cols[start_index+3] if start_index+3 < len(metric_cols) else None
# 
#             trial_num = trial_i + 1  # numbering trials starting at 1
# 
#             # Debug printing
#             print(f"  Trial #{trial_num} => JH={jh_in}, LEWIS={lewis_peak_power}, "
#                   f"NORM={norm_lewis_peak_power}, F={max_force}")
# 
#             # Check if there's already a placeholder row in Results
#             cursor.execute("""
#                 SELECT result_id 
#                 FROM Results
#                 WHERE name = ? AND movement = ? AND pre_post = ? AND Trial_Num = ?;
#             """, (name, movement, pre_post, trial_num))
#             existing_row = cursor.fetchone()
# 
#             if existing_row:
#                 # UPDATE
#                 cursor.execute("""
#                     UPDATE Results
#                     SET JH_IN = ?, 
#                         LEWIS_PEAK_POWER = ?, 
#                         NORM_LEWIS_PEAK_POWER_KG = ?, 
#                         Max_Force = ?
#                     WHERE result_id = ?;
#                 """, (
#                     jh_in,
#                     lewis_peak_power,
#                     norm_lewis_peak_power,
#                     max_force,
#                     existing_row[0]
#                 ))
#             else:
#                 # INSERT new row
#                 cursor.execute("""
#                     INSERT INTO Results (
#                         Trial_Num, pre_post, name, movement, 
#                         JH_IN, LEWIS_PEAK_POWER, NORM_LEWIS_PEAK_POWER_KG, Max_Force
#                     ) VALUES (?, ?, ?, ?, ?, ?, ?, ?);
#                 """, (
#                     trial_num,
#                     pre_post,
#                     name,
#                     movement,
#                     jh_in,
#                     lewis_peak_power,
#                     norm_lewis_peak_power,
#                     max_force
#                 ))
# 
# # Commit once after processing all movements
# conn.commit()
# conn.close()
# 
# print("\nAll trials processed and inserted/updated in the database.")


In [42]:
# import sqlite3
# import pandas as pd
# import numpy as np
# from dash import Dash, dcc, html, Input, Output
# import plotly.graph_objects as go
# 
# # ---------------------------
# # Step 1: Data Loading
# # ---------------------------
# db_path = r'D:/Tramp Test/Tramp_Test2.sqlite'
# conn = sqlite3.connect(db_path)
# 
# # Example query that JOINs with Sessions to get test_date. Adjust as needed:
# query = """
# SELECT
#     R.name,
#     R.movement,
#     R.pre_post,
#     R.JH_IN AS jump_height,
#     R.LEWIS_PEAK_POWER AS peak_power,
#     S.test_date
# FROM Results AS R
# JOIN Participants AS P
#     ON R.name = P.name
# JOIN Sessions AS S
#     ON P.participant_id = S.participant_id
#     AND R.pre_post = S.pre_post
# WHERE R.movement IN ('cmj', 'dj', 'ppu')
# """
# df = pd.read_sql_query(query, conn)
# conn.close()
# 
# # Convert test_date to datetime, if it's not already
# df['test_date'] = pd.to_datetime(df['test_date'], errors='coerce')
# 
# # Drop rows without participant name or pre_post
# df.dropna(subset=['name','pre_post'], inplace=True)
# 
# participants = df['name'].dropna().unique()
# 
# # ---------------------------
# # Step 2: Initialize Dash
# # ---------------------------
# app = Dash(__name__)
# 
# # ---------------------------
# # Step 3: Layout
# # ---------------------------
# # We create 3 rows (CMJ, DJ, PPU), each with 2 columns (Jump Height on the left, Power on the right).
# app.layout = html.Div([
#     html.H1("Pre vs. Post with All Trials + Average Lines"),
# 
#     html.Div([
#         html.Label("Select a Participant:"),
#         dcc.Dropdown(
#             id='participant-dropdown',
#             options=[{'label': p, 'value': p} for p in participants],
#             value=participants[0] if len(participants) else None,
#             clearable=False
#         )
#     ], style={'width': '30%', 'marginBottom': '20px'}),
# 
#     # ---------------------------
#     # CMJ
#     # ---------------------------
#     html.H2("CMJ"),
#     html.Div([
#         dcc.Graph(id='cmj-jh-graph', style={'width': '49%', 'display': 'inline-block'}),
#         dcc.Graph(id='cmj-power-graph', style={'width': '49%', 'display': 'inline-block'})
#     ], style={'display': 'flex'}),
# 
#     # ---------------------------
#     # DJ
#     # ---------------------------
#     html.H2("DJ"),
#     html.Div([
#         dcc.Graph(id='dj-jh-graph', style={'width': '49%', 'display': 'inline-block'}),
#         dcc.Graph(id='dj-power-graph', style={'width': '49%', 'display': 'inline-block'})
#     ], style={'display': 'flex'}),
# 
#     # ---------------------------
#     # PPU
#     # ---------------------------
#     html.H2("PPU"),
#     html.Div([
#         dcc.Graph(id='ppu-jh-graph', style={'width': '49%', 'display': 'inline-block'}),
#         dcc.Graph(id='ppu-power-graph', style={'width': '49%', 'display': 'inline-block'})
#     ], style={'display': 'flex'}),
# 
# ], style={'margin': '20px'})
# 
# 
# # ---------------------------
# # Step 4: Callback
# # ---------------------------
# @app.callback(
#     [
#         Output('cmj-jh-graph','figure'),
#         Output('cmj-power-graph','figure'),
#         Output('dj-jh-graph','figure'),
#         Output('dj-power-graph','figure'),
#         Output('ppu-jh-graph','figure'),
#         Output('ppu-power-graph','figure'),
#     ],
#     [Input('participant-dropdown','value')]
# )
# def update_plots(selected_participant):
#     """For each movement (CMJ, DJ, PPU), create two figures:
#        1) Jump Height
#        2) Peak Power
# 
#        Each figure:
#          - Plots raw points for each session's pre/post
#          - Plots a dashed line from mean(pre) to mean(post)
#          - Each session (test_date) gets a different color
#          - 90% opacity
#          - Black background, gray plot area, white text
#     """
#     dff = df[df['name'] == selected_participant].copy()
#     
#     # For convenience, define a function that builds a single figure for a movement + column.
#     def build_pre_post_figure(movement_label, y_col):
#         """
#         movement_label: 'cmj', 'dj', 'ppu'
#         y_col: 'jump_height' or 'peak_power'
# 
#         Returns a Figure with:
#           - All raw points for pre/post
#           - A dashed line connecting average(pre) -> average(post)
#           - One color per test_date
#         """
#         sub_df = dff[dff['movement'] == movement_label].copy()
#         # Sort by test_date so each test_date has a consistent color order
#         sub_df.sort_values('test_date', inplace=True)
# 
#         # Unique test sessions (dates). We rely on Plotly's default color cycle
#         # so each date gets a different color automatically.
#         unique_dates = sub_df['test_date'].dropna().unique()
# 
#         fig = go.Figure()
# 
#         for session_date in unique_dates:
#             # Filter the data for that session_date
#             sess_mask = (sub_df['test_date'] == session_date)
#             sess_data = sub_df[sess_mask]
#             if sess_data.empty:
#                 continue
# 
#             # Split into pre and post
#             pre_data = sess_data[sess_data['pre_post'].str.lower()=='pre']
#             post_data= sess_data[sess_data['pre_post'].str.lower()=='post']
# 
#             # 1) Raw points for PRE
#             if not pre_data.empty:
#                 fig.add_trace(go.Scatter(
#                     x=['pre']*len(pre_data),
#                     y=pre_data[y_col],
#                     mode='markers',
#                     name=f"{session_date.date()} (pre) pts",
#                     opacity=0.9,
#                     # We'll let Plotly handle the color,
#                     # but you could specify one if you want:
#                     # marker=dict(color='rgba(255,0,0,0.9)'),
#                 ))
# 
#             # 2) Raw points for POST
#             if not post_data.empty:
#                 fig.add_trace(go.Scatter(
#                     x=['post']*len(post_data),
#                     y=post_data[y_col],
#                     mode='markers',
#                     name=f"{session_date.date()} (post) pts",
#                     opacity=0.9
#                 ))
# 
#             # 3) Dashed line from mean(pre) to mean(post), no markers
#             if (not pre_data.empty) and (not post_data.empty):
#                 avg_pre  = pre_data[y_col].mean()
#                 avg_post = post_data[y_col].mean()
#                 if not np.isnan(avg_pre) and not np.isnan(avg_post):
#                     fig.add_trace(go.Scatter(
#                         x=['pre','post'],
#                         y=[avg_pre, avg_post],
#                         mode='lines',
#                         line=dict(dash='dash', width=2),  # dashed line
#                         opacity=0.9,
#                         showlegend=False  # We don't want a separate legend entry for the line
#                     ))
# 
#         # Update layout for dark background, etc.
#         fig.update_layout(
#             paper_bgcolor='black',   # outside the plot
#             plot_bgcolor='grey',     # behind the data
#             font=dict(color='white'),
#             xaxis=dict(title="Pre vs Post", type='category'),
#             yaxis=dict(title=y_col.replace('_',' ').title()),
#             hovermode='closest'  # or 'x unified'
#         )
# 
#         return fig
# 
#     # Build 6 figures
#     cmj_jh_fig    = build_pre_post_figure('cmj', 'jump_height')
#     cmj_power_fig = build_pre_post_figure('cmj', 'peak_power')
#     dj_jh_fig     = build_pre_post_figure('dj',  'jump_height')
#     dj_power_fig  = build_pre_post_figure('dj',  'peak_power')
#     ppu_jh_fig    = build_pre_post_figure('ppu', 'jump_height')
#     ppu_power_fig = build_pre_post_figure('ppu', 'peak_power')
# 
#     return (
#         cmj_jh_fig,
#         cmj_power_fig,
#         dj_jh_fig,
#         dj_power_fig,
#         ppu_jh_fig,
#         ppu_power_fig
#     )
# 
# # ---------------------------
# # Step 5: Run the App
# # ---------------------------
# if __name__ == '__main__':
#     app.run_server(debug=True)
